# Imports

In [1]:
import pandas as pd
import numpy as np
import random
from collections import defaultdict
%matplotlib inline

df = pd.read_csv("salario.csv")
count = defaultdict(int)

In [2]:
def testing():
    return df.groupby(["pais","estado","cidade","genero","data"], axis=0, level=None, as_index=False).count()

def classEquivalence(a,b,c,d,e):
    return df[(df["pais"] == a) & (df["estado"] == b) & (df["cidade"] == c) & (df["genero"] == d) & (df["data"] == e)]
    

In [3]:
def toGeneralize(genero): 
    if genero == True: return random.choice(df.columns.drop('salario').drop('id'))
    return random.choice(df.columns.drop('salario').drop('id').drop('genero'))

def nextGeneralization(i,j, value):
    if(j == "pais"): 
        if(df.at[i,"estado"] != "*"): return nextGeneralization(i, "estado", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 2): count["lugar",i] = 3
    elif(j == "estado"):
        if(df.at[i,"cidade"] != "*"): return nextGeneralization(i, "cidade", value)
        df.at[i, j] = "*"
        if(count["lugar",i] == 1): count["lugar",i] = 2
    elif(j == "cidade"):
        df.at[i, j] = "*"
        if(count["lugar",i] == 0): count["lugar",i] = 1
    elif(j == "genero"):
        df.at[i, j] = "*"
        if(count["genero",i] == 0): count["genero",i] = 1
    elif(j == "data"):
        if(str(value)[0:5] == "**/**"): df.at[i, j] = "**/**/**"
        elif(str(value)[0:2] == "**"): df.at[i, j] = "**/**"  + str((value)[5:8])
        else:
            df.at[i, j] = "**" + str((value)[2:8])
        
        if(count["data",i] < 3): count["data",i] = count["data",i] + 1
    else:
        df.at[i, j] = "erro"
        

In [4]:
def max_hye(col):
    if (col == 'genero'): return 1
    return 3

Na = 3 #n_atributos semi identificadores
def precision():
    summ =  0
    for index, row in df.iterrows():
        for col in df.columns.drop('id').drop('salario'):
                summ += count[col, index]/max_hye(col)

    return 1 - summ/(df.shape[0] * Na)

In [5]:
def nextt(k, genero): #generaliza classe de equivalencia que não satisfaz k-anonimidade
        test = testing()
        test = test[test['salario'] < k]
        
        for index,aux in test.iterrows():
            a = aux['pais']
            b = aux['estado']
            c = aux['cidade']
            d = aux['genero']
            e = aux['data']
        
            
            j = toGeneralize(genero)

            for i in classEquivalence(a,b,c,d,e).index:
                #print(str(i) + " " + str(j))
                nextGeneralization(i, j, aux[j])
            #print("----------------\n\n")
            

            
def nextt2(qtd): #generaliza qtd individuos aleatorios do dataset
    for q in range(qtd):
        i = random.choice(df.index)
        j = toGeneralize(True)
        #print(str(i) + " " + str(j))
        nextGeneralization(i, j, df.loc[i][j])
        #print("----------------\n\n")



def kanonimity(k):
    while testing()[testing()['salario'] < k].shape[0] > 0:
        test = testing()
        test = test[test['salario'] < k]
        qtd_restante = test.shape[0] 
        print(qtd_restante)
        
        #Caso tenha uma classe completamente generalizada que não satisfaz k-anonimidade,
        #devemos, generalizar individuos mesmo que pertencam a alguma classe que satisfaz k-anonimidade.
        #No caso, vamos generalizar 3* a quantidade necessária, de forma aleatória.
        if(qtd_restante == 1 and test['pais'].values[0]=='*' and test['estado'].values[0]=='*' and test['cidade'].values[0]=='*'
        and test['genero'].values[0]=='*' and test['data'].values[0]=='**/**/**'):
            nextt2(3*(k - test.loc[0]['salario']))
        else:
            #Generalizar genero prejudica muito a precisao.
            #Portanto, só será considerado essa possibilidade para as ultimas 50 classes
            if(qtd_restante > 50): 
                nextt(k, False)
            else:
                nextt(k, True)


        

In [6]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(2)


1000
998
944
716
528
392
269
189
132
101
75
55
37
34
29
22
22
20
18
18
14
11
8
7
5
5
4
3
3
2
2
1
1
CPU times: user 20.6 s, sys: 284 ms, total: 20.9 s
Wall time: 20.6 s


In [7]:
precision() #k =2

0.8416666666666675

In [8]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(4)


1000
999
975
848
645
434
305
220
158
111
82
58
43
33
28
24
24
22
21
19
15
10
9
8
8
8
8
7
5
5
5
4
4
3
1
CPU times: user 23.2 s, sys: 250 ms, total: 23.5 s
Wall time: 22.8 s


In [9]:
precision() #k=4

0.816888888888889

In [10]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(8)


1000
999
975
844
649
482
333
237
164
115
88
66
47
40
34
29
27
26
22
21
19
16
14
12
12
10
9
7
5
3
2
2
1
1
1
CPU times: user 23.6 s, sys: 187 ms, total: 23.8 s
Wall time: 23.3 s


In [11]:
precision() #k=8

0.7835555555555554

In [12]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(16)


1000
1000
971
839
661
494
354
267
207
159
127
103
83
67
55
47
38
31
29
26
22
22
18
16
13
11
10
9
7
5
4
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
CPU times: user 25.7 s, sys: 260 ms, total: 26 s
Wall time: 25.4 s


In [13]:
precision() #k=16

0.7378888888888908

In [14]:
%%time
df = pd.read_csv("salario.csv")
count = defaultdict(int)
kanonimity(64)

1000
999
974
836
650
502
371
270
202
162
132
100
79
59
55
41
34
26
23
21
17
15
12
11
11
10
8
8
8
6
5
5
4
2
2
2
2
2
1
3
3
3
2
1
3
2
1
2
2
1
2
2
2
2
2
CPU times: user 26.1 s, sys: 141 ms, total: 26.2 s
Wall time: 25.8 s


In [15]:
precision() #k=64

0.7195555555555578